In [1]:
import torch
import torch.nn as nn
import torchvision as tv
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [2]:
torch.cuda.empty_cache()

In [3]:
writer = SummaryWriter()

In [4]:
class Dataset2class(torch.utils.data.Dataset):
    def __init__(self, path_dir1:str, path_dir2:str):
        super().__init__()
        
        self.path_dir1 = path_dir1
        self.path_dir2 = path_dir2
        
        self.list_dir1 = sorted(os.listdir(path_dir1))
        self.list_dir2 = sorted(os.listdir(path_dir2))
        
    def __len__(self):
        return len(self.list_dir1) + len(self.list_dir2)
    
    def __getitem__(self, idx):
        
        if idx < len(self.list_dir1):
            class_id = 0
            img_path = os.path.join(self.path_dir1, self.list_dir1[idx])
        else:
            class_id = 1
            idx -= len(self.list_dir1)
            img_path = os.path.join(self.path_dir2, self.list_dir2[idx])
            
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32)/255.0
        img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_AREA)
        img = img.transpose((2, 0, 1))
        
        t_img = torch.from_numpy(img)
        
        t_class_id = torch.tensor(class_id)
        
        return (t_img, class_id)

In [5]:
data_dogs_path = 'pathdata_cats_path = 'path
dataset_catvsdog = Dataset2class(data_cats_path, data_dogs_path)

In [6]:
train_ds, val_ds = torch.utils.data.random_split(dataset_catvsdog, [0.8, 0.2])

In [7]:
batch_size = 32
shuffle = True

train_loader = torch.utils.data.DataLoader(
    dataset=train_ds,
    batch_size=batch_size,
    shuffle=shuffle,
    drop_last=True
)

val_loader = torch.utils.data.DataLoader(
    dataset=val_ds,
    batch_size=batch_size,
    shuffle=shuffle,
    drop_last=False
)

In [8]:
class VGG16(nn.Module):
    def __init__(self, out_nc):
        super().__init__()
        
        self.act = nn.ReLU(inplace=True)
        
        self.maxpool = nn.MaxPool2d(2, 2)
        
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        
        self.flat = nn.Flatten()
        
        self.fc1 = nn.Linear(7*7*512, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, out_nc)
        
        self.BatchNorm = nn.BatchNorm1d(4096)
        
    def forward(self, x):
        out = self.conv1_1(x)
        out = self.act(out)
        out = self.conv1_2(out)
        out = self.act(out)
        
        out = self.maxpool(out)

        out = self.conv2_1(out)
        out = self.act(out)
        out = self.conv2_2(out)
        out = self.act(out)
        
        out = self.maxpool(out)
        
        out = self.conv3_1(out)
        out = self.act(out)
        out = self.conv3_2(out)
        out = self.act(out)
        out = self.conv3_3(out)
        out = self.act(out)
        
        out = self.maxpool(out)
        
        out = self.conv4_1(out)
        out = self.act(out)
        out = self.conv4_2(out)
        out = self.act(out)
        out = self.conv4_3(out)
        out = self.act(out)
        
        out = self.maxpool(out)
        
        out = self.conv5_1(out)
        out = self.act(out)
        out = self.conv5_2(out)
        out = self.act(out)
        out = self.conv5_3(out)
        out = self.act(out)
        
        out = self.maxpool(out)
        
        out = self.flat(out)
        
        out = self.fc1(out)
        out = self.BatchNorm(out)
        out = self.act(out) 
        out = self.fc2(out)
        out = self.BatchNorm(out)
        out = self.act(out)
        out = self.fc3(out)
        
        return out

In [9]:
model = VGG16(2)
# model = torch.load('4XConv+4XMaxpool+2Xlieear_Model.pth')

In [10]:
loss_fn = nn.CrossEntropyLoss()

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    total_loss=0
    correct=0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        
        X, y = X.cuda(), y.cuda()
        
        pred = model(X)
        loss = loss_fn(pred, y)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        tensor_board.add_scalar('Train_loss', total_loss, epoch)
        tensor_board.add_scalar('Test_accuracy', correct / size, epoch)

def test_loop(dataloader, model, loss_fn, max_test_accuracy, epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.cuda(), y.cuda()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    
    tensor_board.add_scalar('Test_loss', test_loss, epoch)
    tensor_board.add_scalar('Test_accuracy', correct / size, epoch)
    
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    
    if correct < max_test_accuracy:
        return max_test_accuracy
    else:
        model_file_name = '4XConv+4XMaxpool+2Xlieear_Model.pth'
        print(f'New Accuracy record: {(100*max_test_accuracy):>0.1f}% ----> '
              f'{(100*correct):>0.1f}% saving new model {model_file_name}')
        torch.save(model, model_file_name)
        return correct

In [12]:
learning_rate = 0.001

tensor_board = SummaryWriter(comment = f"Batch size={batch_size} Learning rate={learning_rate} Shuffle={shuffle}")

optimizer = torch.optim.AdamW(params = model.parameters(), lr=learning_rate)

In [13]:
model.to('cuda')
loss_fn.to('cuda')

CrossEntropyLoss()

In [14]:
epochs = 5
max_test_accuracy = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer, epoch=t+1)
    max_test_accuracy = test_loop(val_loader, model, loss_fn, max_test_accuracy, epoch=t+1)
print("Done!")
tensor_board.close()

Epoch 1
-------------------------------
loss: 0.083552  [   32/19957]
loss: 0.078578  [ 3232/19957]
loss: 0.038333  [ 6432/19957]
loss: 0.206599  [ 9632/19957]
loss: 0.125698  [12832/19957]
loss: 0.010968  [16032/19957]
loss: 0.072404  [19232/19957]
Test Error: 
 Accuracy: 96.4%, Avg loss: 0.088371 

New Accuracy record: 0.0% ----> 96.4% saving new model 4XConv+4XMaxpool+2Xlieear_Model.pth
Epoch 2
-------------------------------
loss: 0.026345  [   32/19957]
loss: 0.011585  [ 3232/19957]
loss: 0.023104  [ 6432/19957]
loss: 0.009080  [ 9632/19957]
loss: 0.024538  [12832/19957]
loss: 0.006950  [16032/19957]
loss: 0.025009  [19232/19957]
Test Error: 
 Accuracy: 96.7%, Avg loss: 0.087901 

New Accuracy record: 96.4% ----> 96.7% saving new model 4XConv+4XMaxpool+2Xlieear_Model.pth
Epoch 3
-------------------------------
loss: 0.025097  [   32/19957]
loss: 0.044253  [ 3232/19957]
loss: 0.035673  [ 6432/19957]
loss: 0.019586  [ 9632/19957]
loss: 0.030367  [12832/19957]
loss: 0.225238  [16032/